<a href="https://colab.research.google.com/github/taka-onohara/google-colab/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','avh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Y = rgdpna, K = rkna, L = emp * avh, alpha = 1 - labsh, A = (the rest)
data['Y'] = data['rgdpna']
data['K'] = data['rkna']
data['L'] = data['emp'] * data['avh']
data['alpha'] = 1 - data['labsh']
data['A'] = data['Y'] / ((data['K'] ** data['alpha']) * (data['L'] ** data['labsh']))

data['test'] = data['A'] * (data['K'] ** data['alpha']) * (data['L'] ** data['labsh'])
data.drop(data.columns[[2,3,4,5,6]], axis=1, inplace=True)
# print(data.head(10))

# y = Y/L, k = K/L, ln(y) = ln(A) + alpha * ln(k)
data['ln(y)'] = np.log(data['Y'] / data['L'])
data['ln(A)'] = np.log(data['A'])
data['ln(k)'] = np.log(data['K'] / data['L'])

data['test1'] = data['ln(A)'] + data['alpha'] * data['ln(k)']
# print(data.head(10))

data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
    growth_rate = x['ln(y)'].diff() * 100,
    TFP_growth = x['ln(A)'].diff() * 100,
    capital_deepening = (x['alpha'] * x['ln(k)']).diff() * 100
)).reset_index(drop=True).dropna()

data['test2'] = data['TFP_growth'] + data['capital_deepening']
#print(data.head(10))

data.rename(columns = {'alpha':'capital_share'}, inplace = True)
data['TFP_share'] = 1 - data['capital_share']

summary = data.groupby('countrycode').agg({'growth_rate': 'mean',
                                           'TFP_growth':'mean',
                                           'capital_deepening':'mean',
                                           'TFP_share':'mean',
                                           'capital_share':'mean'})
print('---summary---')
print(summary)

overall = summary.mean()
print('---overall average---')
print(overall)

---summary---
             growth_rate  TFP_growth  capital_deepening  TFP_share  \
countrycode                                                          
CAN             1.117384    1.677697          -0.560312   0.653810   
DEU             1.073874    1.409886          -0.336012   0.629270   
FRA             1.087678    0.247012           0.840665   0.618806   
GBR             1.149943   -1.598218           2.748162   0.587007   
ITA             0.285631   -0.256089           0.541721   0.515165   
JPN             1.221666    2.118842          -0.897175   0.569399   
USA             1.658237    1.355163           0.303074   0.606740   

             capital_share  
countrycode                 
CAN               0.346190  
DEU               0.370730  
FRA               0.381194  
GBR               0.412993  
ITA               0.484835  
JPN               0.430601  
USA               0.393260  
---overall average---
growth_rate          1.084916
TFP_growth           0.707756
capital_deep